# 1. import all necessary imports

In [1]:
from tensorflow.keras.models import model_from_json
import numpy as np
import urllib.request
import pickle
import os
import gzip

# 2. load encoder weights

In [2]:
def load_model(name):
    ''' Load any model'''
    filename_json = name + ".json"
    filename_h5 = name + ".h5"
    # load json and create model
    json_file = open(filename_json, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    # load weights into new model
    loaded_model.load_weights(filename_h5)
    print("Loaded model " + name + " from disk")
    return loaded_model

# obs_prop, obs_touch, obs_voxel
encoder_3x = load_model('models/touchsensors_multisensory_sens_proprio_voxel_touch_enc_mean_ProprioVoxelTouch111')
# obs_prop, obs_touch
encoder_2x = load_model('models/touchsensors_multisensory_sens_proprio_voxel_touch_enc_mean_ProprioVoxelTouch101')

Loaded model models/touchsensors_multisensory_sens_proprio_voxel_touch_enc_mean_ProprioVoxelTouch111 from disk
Loaded model models/touchsensors_multisensory_sens_proprio_voxel_touch_enc_mean_ProprioVoxelTouch101 from disk


# 3. load data into the variables

In [3]:
def download(url, location):    
    if not os.path.isfile(location):
        urllib.request.urlretrieve(url, location)


def norm_min_max(data, min = 0.0, max = 1.0, axis = None):
    ''' Scales the input data to the given minimum and maximum value
    
    data (np.array): The input data
    min     (float): Smallest value in data is scaled to this value
    max     (float): Biggest value in data is scaled to this value
    
    returns the normalized values
    '''
    data = data - np.min(data, axis = axis)
    data = data / np.max(data, axis = axis)
    data = (data * (max - min)) + min
    return data

def norm_min_max_native_range(data, native_min = -np.pi/2.0, native_max = np.pi/2.0, min = 0.0, max = 1.0):
    ''' Scales the input data to the given the native range of values to the minimum and maximum value
    
    data    (np.array): The input data
    native_min (float): Lower bound in the native data range
    native_max (float): Upper bound in the native data range
    min        (float): Smallest value in data is scaled to this value
    max        (float): Biggest value in data is scaled to this value
    
    returns the normalized values
    '''
    if (np.min(data) < native_min) or (np.max(data) > native_max):
        raise Exception("data outside of native range")
    data = (data - native_min) / native_max # scale to 0 ... 1 of native range
    data = (data * (max - min)) + min       # scale to desired range
    return data

filenames = ["multisensory_data_ep_162615.pgz", "multisensory_data_ep_162615.pgz", "multisensory_data_ep_162628.pgz", "multisensory_data_ep_162643.pgz", "multisensory_data_ep_162658.pgz", "multisensory_data_ep_162713.pgz", "multisensory_data_ep_162728.pgz", "multisensory_data_ep_162742.pgz", "multisensory_data_ep_162757.pgz", "multisensory_data_ep_162812.pgz", "multisensory_data_ep_162827.pgz"]
sens_proprio = None
sens_pos_quat = None
sens_touch = None
sens_voxel = None
for filename in filenames:
    url = "https://github.com/ndrwmlnk/baselines/blob/touchsensors_multisensory_data/HandManipulateBlockTouchSensors_multisensory_data/" + filename + "?raw=true"
    location = "/tmp/" + filename
    download(url, location)

    # Each file is a single episode (100 steps) and contains a dict:
    # senses = dict(
    # sens_proprio,  # 24 joint vaplues for - 24 degrees of freedom
    # sens_touch,  # 75 binary touch sensors: 0 - no touch, 1 - touch detected
    # sens_pos_quat,  # 7 value vectors: first 3 - XYZ of the cube, last 4 - quat values
    # sens_voxel  # 16x16x16 voxel space, cube shape is alwas in the center, cube size 8x8x8, check these links [1, 2] for visualization
    # )
    

    # Read '*.pgz' files:
    file = location
    with gzip.open(file, 'r') as f:
       data = pickle.load(f)
    
    if sens_proprio is None:
        sens_proprio = np.array(data["sens_proprio"])
    else:
        sens_proprio = np.append(sens_proprio, np.array(data["sens_proprio"]), axis=0)
    if sens_pos_quat is None:
        sens_pos_quat = np.array(data["sens_pos_quat"])
    else:
         sens_pos_quat = np.append(sens_pos_quat, np.array(data["sens_pos_quat"]), axis=0)
    if sens_touch is None:
        sens_touch = np.array(data["sens_touch"])
    else:
         sens_touch = np.append(sens_touch, np.array(data["sens_touch"]), axis=0)
    if sens_voxel is None:
        sens_voxel = np.array(data["sens_voxel"])
    else:
        sens_voxel = np.append(sens_voxel, np.array(data["sens_voxel"]), axis=0)


# sens_proprio
sens_proprio_norm = norm_min_max(sens_proprio, axis = 0)

# sens_voxel (float0..1 1x16x16x16 or 1x4096 if flatten)
sens_voxel_norm = sens_voxel # already normalizes?
sens_voxel_flat_norm = sens_voxel_norm.reshape((len(sens_voxel_norm), np.prod(sens_voxel_norm.shape[1:])))
sens_voxel_norm_flatten = sens_voxel_norm.reshape((sens_voxel_norm.shape[0], -1))

# sens_touch (binary 1x75)
sens_touch_norm = sens_touch # already normalizes?

#################### a single sample of * data ###########################################################
idx = 42
obs_prop = sens_proprio_norm[[idx]]
obs_touch = sens_touch_norm[[idx]]
obs_voxel = sens_voxel_norm[[idx], :, :, :, None]

# 4. Predict

In [4]:
obs_encoded_3x = encoder_3x.predict([obs_prop, obs_voxel, obs_touch])
obs_encoded_2x = encoder_2x.predict([obs_prop, obs_touch])